In [1]:
import datetime as dt
import numpy
import pandas as pd
import metview
import calendar
from ecmp.MediaPy.StvlRetrieve import StvlRetrieveToGeopoints

ecmp (formerly known as ECMWF Metpy) version 1.2.4 (build Tue  7 Jan 14:12:14 UTC 2020 #1.2.4).
/usr/local/apps/python3/3.6.8-01/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
# Define parameters of interest and related accumulation periods
# https://confluence.ecmwf.int/display/VER/stvl+service
# Parameters: total precipitation in the last 24h (mm), 2m temperature (K), relative humidity (%), wind speed (m/s)
parameters = ["tp", "2t", "r", "10ff"]
# Accumulation period (in seconds)
obs_period = [24*3600, 0, 0, 0, 0]

In [ ]:
for month in range(1, 13):
    # Find last day of month to set the time window to 1 calendar month
    month_as_string = str(format(month, '02d'))
    last_of_month = str(calendar.monthrange(2017, month)[1])    
    period = pd.date_range('2017-' + month_as_string + '-01T00:00:00.000Z',
                           '2017-' + month_as_string + '-' + last_of_month + 'T23:00:00.000Z',
                           freq = 'H')
    print(period)
    
    # Initialise empty list
    appended_data = []
    
    # Retrieve observations from STVL
    for param, obst in zip(parameters, obs_period):
        print(param, obst)
        for geo in StvlRetrieveToGeopoints(table = "observation",
                                           parameter = param,
                                           period = obst,
                                           reference_datetimes = period):
            data = geo.to_dataframe()
            data['param'] = param
            # store DataFrame in list
            appended_data.append(data)
        
    # Concatenate list to dataframe
    appended_data = pd.concat(appended_data)
    # Write DataFrame to a csv file
    appended_data.to_csv('/hugetmp/SYNOP/Observations_2017' + month_as_string + '.csv')

DatetimeIndex(['2017-01-01 00:00:00+00:00', '2017-01-01 01:00:00+00:00',
               '2017-01-01 02:00:00+00:00', '2017-01-01 03:00:00+00:00',
               '2017-01-01 04:00:00+00:00', '2017-01-01 05:00:00+00:00',
               '2017-01-01 06:00:00+00:00', '2017-01-01 07:00:00+00:00',
               '2017-01-01 08:00:00+00:00', '2017-01-01 09:00:00+00:00',
               ...
               '2017-01-31 14:00:00+00:00', '2017-01-31 15:00:00+00:00',
               '2017-01-31 16:00:00+00:00', '2017-01-31 17:00:00+00:00',
               '2017-01-31 18:00:00+00:00', '2017-01-31 19:00:00+00:00',
               '2017-01-31 20:00:00+00:00', '2017-01-31 21:00:00+00:00',
               '2017-01-31 22:00:00+00:00', '2017-01-31 23:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=744, freq='H')
tp 86400
2t 0
